In [2]:
import pandas as pd
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.model_selection import  GridSearchCV, RandomizedSearchCV
from KSVM import KSVC, KSVR
from KANN import KANNC,KANNR
from sklearn.model_selection import train_test_split,KFold, cross_val_score,ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
import numpy as np
from time import time
from multiprocessing.pool import Pool
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error

In [95]:
def normalizerTransform(x,nm):       
    if(nm=="Min_Max"):#mix-max
        scaler=MinMaxScaler()
        scaler.fit(x)
        return scaler.transform(x)
    if(nm=="Normalizer"):#normalizer
        scaler=Normalizer()
        scaler.fit(x)
        return scaler.transform(x)
    
    if(nm=="Standard"):#standard
        scaler=StandardScaler()
        scaler.fit(x)
        return scaler.transform(x)    
    return x

def runHoldout(X,y,params,iteration=1):
    clf=None 
    results=[]
   
    for r in params:
        
        print(r['model'],r['normalizer'],r['kernel'],'\n')
        if(r['model']=='KSVR'):
            if(r['kernel']=='rquadratic'):
                clf=KSVR(C=r['C'],coef0=r['coef0'],gamma=1,kernel=r['kernel'])
            else:
                clf=KSVR(C=r['C'],gamma=r['gamma'],kernel=r['kernel'])
        else:
            if(r['kernel']=='rquadratic'):
                clf=KANNR(early_stopping=True, max_iter=5000,gamma=1,coef0=r['coef0'],kernel=r['kernel'])
            else:
                clf=KANNR(early_stopping=True, max_iter=5000,gamma=r['gamma'],kernel=r['kernel'])
        Xn=normalizerTransform(X,r['normalizer'])
        vsae=[]
        vmae=[]
        vrmse=[]
        vr2=[]
        for i in range(iteration):
            X_train, X_test, y_train, y_test = train_test_split(Xn, y, test_size=1/3)
            clf.fit(X_train,y_train)
            y_pred=clf.predict(X_test)
            n=y_pred.shape[0]
            mae=mean_absolute_error(y_test,y_pred)
            mse=mean_squared_error(y_test,y_pred)
            rmse=mse**.5
            sae=n*mae                        
            r2=clf.score(X_test,y_test)
            vsae.append(sae)
            vmae.append(mae)
            vrmse.append(rmse)
            vr2.append(r2)
            mresult={'model':r['model'],'normalizer':r['normalizer'],'kernel': r['kernel'],
                'sae':sae,'mae':mae,'rmse':rmse,'r2':r2}
            results.append(mresult)
     
        vsae=np.array(vsae)
        vmae=np.array(vmae)
        vrmse=np.array(vrmse)
        vr2=np.array(vr2)
        #mr={'model':r['model'],'normalizer':r['normalizer'],'kernel': r['kernel'],
                #'sae':vsae,'mae':vmae,'rmse':rmse,'r2':vr2}
        #print(mr)   
        print("sae",vsae.mean(),vsae.std())
        print("mae",vmae.mean(),vmae.std())
        print("rmse",vrmse.mean(),vrmse.std())
        print("r2",vr2.mean(),vr2.std())
        print()
        
        
    df=pd.DataFrame(results)
    display(df)
    df.to_csv('hold_out.csv')
                

# Landsat holdout test

In [4]:
dff=pd.read_csv('landsat_model.csv')        
nr=dff.shape[1]
Xl=dff.iloc[:,0:nr-1]
yl=dff.iloc[:,nr-1:nr]    
yl=yl.values.ravel()
print(Xl.shape, yl.shape)
dff.head()

(434, 9) (434,)


,latitude,longitude,band1,band2,band3,band4,band5,band6,band7,value
0,-8789850.0,184950.0,0.105113,0.082716,0.052756,0.315520,0.132641,296.178607,0.044586,203.4
1,-8784900.0,174600.0,0.104840,0.085732,0.053527,0.271463,0.090247,295.969710,0.027292,198.3
2,-8775000.0,164700.0,0.103468,0.079147,0.048932,0.248619,0.078593,296.842689,0.022098,203.1
3,-8775000.0,169650.0,0.108849,0.090665,0.055715,0.338889,0.128227,296.123927,0.039798,199.1
4,-8775000.0,174600.0,0.096526,0.071028,0.042247,0.194788,0.054468,294.407491,0.017712,196.3


Landsat

KSVR

Min Max 
16.798540830612183 0.9254600724210832 {'kernel': 'rquadratic', 'coef0': 0.1, 'C': 100.0}

14.107083797454834 0.9100940227979385 {'kernel': 'triangle', 'gamma': 100.0, 'C': 1000.0}

Normalizer

29.072359561920166 0.8520211520434284 {'kernel': 'can', 'gamma': 10.0, 'C': 100.0}


Standard 

13.600720405578613 0.9304369490472607 {'kernel': 'triangle', 'gamma': 100.0, 'C': 10000.0}

------------------------------------------------------------------------------------------
KANNR

Min Max 
56.00196695327759 0.7335709082387473 {'kernel': 'rbf', 'gamma': 1.0, 'C': 10.0}

26.77895212173462 0.7722875878844598 {'kernel': 'radial_basic', 'gamma': 10.0, 'C': 10.0}

Normalizer
34.672629833221436 0.6676965674816054 {'kernel': 'can', 'gamma': 10.0, 'C': 10000.0}

Standard
27.66696286201477 0.8452109187984763 {'kernel': 'hyperbolic', 'gamma': 0.1, 'C': 10.0}

===========================================================================================

In [16]:
paramsl=[{'model':'KSVR','normalizer':'Min_Max','kernel': 'rquadratic', 'coef0': 1, 'C': 1000.0},
         {'model':'KSVR','normalizer':'Standard','kernel': 'rbf', 'gamma': 0.01, 'C': 10000.0},
         {'model':'KSVR','normalizer':'Standard','kernel': 'triangle', 'gamma': 100.0, 'C': 1000.0}]



In [86]:
print('Landsat')
runHoldout(Xl,yl,paramsl,iteration=1000)



Landsat
KSVR Min_Max rquadratic 

{'model': 'KSVR', 'normalizer': 'Min_Max', 'kernel': 'rquadratic', 'sae': array([534.40708044, 263.96670981, 281.68415764, 273.89716316,
       257.05892713, 330.28843619, 342.06713517, 300.83199   ,
       297.08165335, 280.42787433, 279.44801443, 299.86932603,
       271.73853185, 281.20218953, 283.48535969, 324.70507082,
       341.26015657, 282.50157982, 307.55432761, 292.52672411,
       299.37952773, 257.7035338 , 320.3235181 , 317.75053669,
       367.18514023, 303.10605829, 264.46175555, 364.00707477,
       382.72918173, 264.68414811, 317.0172586 , 297.87903199,
       286.42681182, 278.20941467, 291.11493031, 306.10029039,
       327.51376189, 309.54485413, 240.19562325, 315.94171191,
       297.72222153, 273.46667838, 294.59142884, 292.59799126,
       351.67413303, 312.75778899, 268.18697033, 254.49830764,
       257.32405005, 240.74275121, 297.00978409, 279.77375747,
       283.55939102, 326.15873619, 338.91555835, 340.3056058 ,
       294

{'model': 'KSVR', 'normalizer': 'Standard', 'kernel': 'rbf', 'sae': array([581.11676541, 650.85597907, 537.29018027, 572.05130737,
       548.15480655, 587.42172488, 603.99586242, 528.1671051 ,
       551.86418784, 549.09606469, 600.33379618, 613.4151181 ,
       576.36084895, 683.73862898, 642.14730966, 601.93218743,
       558.34183097, 549.22365184, 565.14774928, 585.52577235,
       567.31113531, 507.20545366, 593.00149924, 615.33310689,
       622.6825    , 552.25588415, 605.57143944, 575.34961323,
       658.18895333, 644.02859764, 548.51079394, 682.44359816,
       577.00390942, 639.92766706, 578.14447239, 570.50264386,
       578.88560105, 569.68853638, 600.24682315, 717.08345804,
       558.79156901, 616.73170829, 547.03449888, 605.86417431,
       638.11868555, 564.75215648, 680.61995737, 594.92316358,
       645.2821145 , 629.59745197, 558.3223967 , 597.47898982,
       695.2610155 , 627.04609865, 650.59346016, 554.9924082 ,
       689.03187401, 577.08539647, 631.62073653, 5

{'model': 'KSVR', 'normalizer': 'Standard', 'kernel': 'triangle', 'sae': array([546.98557709, 222.9931384 , 240.50512641, 185.79562644,
       255.634068  , 235.75223587, 218.55465808, 182.42063345,
       211.8421848 , 212.96761985, 196.85869571, 238.73231285,
       225.66104342, 220.75110072, 218.64060047, 268.03740149,
       192.10513535, 218.68304564, 230.46122078, 225.58473524,
       179.47811856, 231.6925374 , 177.91987631, 190.59620143,
       226.11452633, 205.22539063, 179.31902247, 213.69267162,
       226.35475783, 183.40504778, 198.4478794 , 246.91518333,
       260.03623053, 252.20013392, 224.11380787, 221.61368299,
       232.64712126, 220.22044691, 267.24655463, 156.35614301,
       234.66718398, 243.26927115, 233.09722268, 214.21386974,
       215.73000395, 173.21249173, 114.67049207, 278.22739015,
       231.25028086, 208.78464043, 238.26951422, 167.15797191,
       201.49618571, 223.98755341, 193.65441273, 253.05817797,
       181.81074377, 162.90935342, 204.962330

,model,normalizer,kernel,sae,mae,rmse,r2
0,KSVR,Min_Max,rquadratic,534.407080,3.685566,5.365838,0.906234
1,KSVR,Min_Max,rquadratic,263.966710,1.820460,3.209146,0.968380
2,KSVR,Min_Max,rquadratic,281.684158,1.942649,3.644274,0.953686
3,KSVR,Min_Max,rquadratic,273.897163,1.888946,3.547480,0.957021
4,KSVR,Min_Max,rquadratic,257.058927,1.772820,3.596379,0.957399
...,...,...,...,...,...,...,...
2995,KSVR,Standard,triangle,215.662211,1.487326,3.056037,0.968674
2996,KSVR,Standard,triangle,207.479119,1.430890,3.127504,0.961481
2997,KSVR,Standard,triangle,256.879839,1.771585,3.578767,0.956242
2998,KSVR,Standard,triangle,197.952072,1.365187,2.993068,0.967458


In [68]:
Xnl=normalizerTransform(Xl,'Standard')
#kernel= 'rquadratic', coef0= 0.1, C= 100.0
#C=10000.,gamma=100.,kernel='triangle'
clfl=KSVR(kernel= 'triangle', gamma= 100.0, C= 1000.0)
Xfl, Xtl, yfl, ytl = train_test_split(Xnl, yl, test_size=1/3)
clfl.fit(Xfl,yfl)
stl=clfl.score(Xtl,ytl)
sfl=clfl.score(Xfl,yfl)
sal=clfl.score(Xnl,yl)
print(stl,sfl,sal)

0.9216362177248467 0.9926572482924729 0.9678623840408875


In [87]:
dfm=pd.read_csv('modis_model.csv')        
nr=dfm.shape[1]
Xm=dfm.iloc[:,0:nr-1]
ym=dfm.iloc[:,nr-1:nr]    
ym=ym.values.ravel()
print(Xm.shape, ym.shape)
dfm.head()

(445, 9) (445,)


,latitude,longitude,band1,band2,band3,band4,band5,band6,band7,value
0,-8789850.0,184950.0,3296.903053,4486.123350,3395.320132,3448.007013,4321.515264,3236.024752,2072.205033,203.4
1,-8784900.0,174600.0,2723.544879,3882.381574,2847.123764,2902.167260,3775.932780,2743.972716,1732.732701,198.3
2,-8775000.0,169650.0,3254.548374,4358.613821,3360.056504,3400.073577,4235.783333,3224.044715,2056.696341,199.1
3,-8775000.0,174600.0,3180.592105,4523.587582,3305.417352,3344.582237,4318.231908,3244.020970,2032.567434,196.3
4,-8775000.0,184950.0,3307.844581,4718.300204,3427.576687,3472.932106,4445.462986,3319.962372,2059.508384,196.9


Modis

KSVR
Min Max 
4.798241376876831 0.9078599031147738 {'kernel': 'rbf', 'gamma': 1.0, 'C': 10000.0}

32.15599846839905 0.9090508712231206 {'kernel': 'radial_basic', 'gamma': 1.0, 'C': 1000.0}

Normalizer

29.278666734695435 0.6675007569418857 {'kernel': 'can', 'gamma': 100.0, 'C': 10.0}

26.634699821472168 0.64613060797684 {'kernel': 'tru', 'gamma': 0.001, 'C': 100.0}

14.26632046699524 0.7013525862263811 {'kernel': 'triangle', 'gamma': 0.01, 'C': 1000.0}

Standard
18.095561027526855 0.895021626040627 {'kernel': 'rquadratic', 'coef0': 1.0, 'C': 1000.0}

14.01233434677124 0.9010610414892459 {'kernel': 'triangle', 'gamma': 1000.0, 'C': 10000.0}

-----------------------------------------------------------------------------------------

KANNR
Min Max 

22.28480076789856 0.6558240357445315 {'kernel': 'radial_basic', 'gamma': 1.0, 'C': 1000.0}

Normalizer

Pesimos

Standard

31.57520580291748 0.7610117791800818 {'kernel': 'hyperbolic', 'gamma': 10.0, 'C': 1000.0}


In [20]:
paramsm=[{'model':'KSVR','normalizer':'Min_Max','kernel': 'rbf', 'gamma': 10.0, 'C': 100.0},
         {'model':'KSVR','normalizer':'Standard','kernel': 'rquadratic', 'coef0': 1.0, 'C': 1000.0},
         {'model':'KSVR','normalizer':'Min_Max','kernel': 'rquadratic', 'coef0': 0.1, 'C': 1000.0},
         ]


In [88]:
print('Modis')
runHoldout(Xm,ym,paramsm,iteration=1000)

Modis
KSVR Min_Max rbf 

{'model': 'KSVR', 'normalizer': 'Min_Max', 'kernel': 'rbf', 'sae': array([516.55033299, 522.63041628, 511.51996486, 534.93611262,
       574.07907891, 577.42323381, 537.72015554, 523.99801339,
       463.72122483, 491.82871112, 621.03171211, 599.24501197,
       477.53778989, 496.54504518, 444.72221218, 525.98446002,
       447.58174068, 557.81030219, 554.1735064 , 531.9377666 ,
       465.79190256, 518.03027917, 500.85501423, 425.59812589,
       535.41073874, 560.11852394, 544.63273787, 481.5424543 ,
       540.40767773, 572.50764611, 471.78172773, 438.77662902,
       479.54440857, 649.08448077, 516.50312398, 441.06796175,
       577.23984675, 564.78432323, 576.31733087, 586.85695566,
       534.32333674, 524.5532658 , 494.82847912, 542.41500813,
       548.41744694, 521.14712519, 480.85954634, 552.99451176,
       476.56536181, 583.19814576, 564.46494574, 488.78062507,
       453.66162887, 551.46571177, 588.31399559, 589.41602678,
       526.21428999, 517.1

{'model': 'KSVR', 'normalizer': 'Standard', 'kernel': 'rquadratic', 'sae': array([572.36165874, 170.45453299, 189.75328903, 239.6590283 ,
       255.5873719 , 274.44785555, 197.81879886, 185.8015258 ,
       195.60481992, 161.36064569, 205.30250656, 212.64218924,
       181.91564948, 169.10131997, 240.23446521, 192.68135358,
       186.59818216, 192.95228725, 194.85324274, 213.32287569,
       149.44108112, 173.36248258, 195.59452537, 201.16495042,
       202.41325259, 193.03090777, 240.56040016, 154.15065409,
       185.35344411, 166.86119723, 212.40940964, 174.92164451,
       218.91183663, 227.03423279, 205.95888577, 189.50091776,
       221.29463148, 233.14730053, 157.16110089, 225.27736186,
       220.30295978, 202.46596848, 186.38534421, 239.73439785,
       184.37706956, 120.5422448 , 202.31979381, 217.07020834,
       219.88181441, 213.08199559, 193.91793219, 183.26073402,
       247.7914603 , 227.8015295 , 204.53877013, 220.7768396 ,
       210.1683179 , 248.39068741, 184.2911

{'model': 'KSVR', 'normalizer': 'Min_Max', 'kernel': 'rquadratic', 'sae': array([532.04720564, 152.2463815 , 180.66806209, 138.05552018,
       194.92969597, 190.47477519, 156.89758295, 163.57194825,
       195.61280115, 199.71497374, 211.75162042, 170.56733095,
       164.60535637, 181.63562584, 175.48296364, 184.53993615,
       189.50355202, 166.03712111, 168.65403247, 213.75360922,
       131.71438753, 179.7754517 , 220.11583028, 179.06809168,
       162.73369483, 214.16909999, 216.60396042, 167.6775752 ,
       185.1144258 , 198.03826461, 155.58253621, 180.51617305,
       169.64809409, 186.72681155, 208.16552135, 155.44972957,
       236.69675171, 205.66265711, 214.09196403, 164.36825943,
       217.63887004, 215.53163605, 163.86596105, 206.75887747,
       175.79802651, 160.78288071, 190.90373824, 155.52092996,
       151.29695863, 178.4094021 , 201.48069714, 172.08704834,
       213.31549343, 183.21251831, 156.17545516, 234.24101713,
       164.49898089, 210.91357402, 215.41181

,model,normalizer,kernel,sae,mae,rmse,r2
0,KSVR,Min_Max,rbf,516.550333,3.466781,5.196624,0.913201
1,KSVR,Min_Max,rbf,522.630416,3.507587,5.241658,0.899000
2,KSVR,Min_Max,rbf,511.519965,3.433020,5.705042,0.885215
3,KSVR,Min_Max,rbf,534.936113,3.590175,5.642333,0.889656
4,KSVR,Min_Max,rbf,574.079079,3.852880,5.921333,0.886656
...,...,...,...,...,...,...,...
2995,KSVR,Min_Max,rquadratic,215.268008,1.444752,3.368135,0.962841
2996,KSVR,Min_Max,rquadratic,196.465467,1.318560,3.459971,0.960024
2997,KSVR,Min_Max,rquadratic,169.967808,1.140724,3.144671,0.970460
2998,KSVR,Min_Max,rquadratic,129.713435,0.870560,2.271936,0.982213


In [92]:
Xnm=normalizerTransform(Xm,'Standard')
#kernel= 'rquadratic', coef0= 0.1, C= 100.0
#C=10000.,gamma=100.,kernel='triangle'
clfm=KSVR(kernel= 'rquadratic', coef0=0.1 , C= 1000.0)
Xfm, Xtm, yfm, ytm = train_test_split(Xnm, ym, test_size=0.2)
clfm.fit(Xfm,yfm)
stm=clfm.score(Xtm,ytm)
sfm=clfm.score(Xfm,yfm)
sam=clfm.score(Xnm,ym)
print(stm,sfm,sal)

0.9171849451705137 0.999966980274499 0.9678623840408875
